In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import glob
import json
import copy

In [2]:
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']

# bbox annotation

In [3]:

names = os.listdir("../../inpainting/0_original_video")
bbox = {}
data = "../data"
bbox_json = "../bbox_json"

if not os.path.isdir(data):
    os.makedirs(data)
if not os.path.isdir(bbox_json):
    os.makedirs(bbox_json)
confirm_bbox = False
dataset_index = 0

continue_flag = True

for name in names:
    video_name= "../../inpainting/0_original_video/{names}".format(names=name)
    
    
    for deg in degrees:
        
        tracker = cv2.TrackerCSRT_create()  # 創建追蹤器
        tracking = False                    # 設定 False 表示尚未開始追蹤
        
        
        video_name_deg = os.path.join(video_name,deg)
        video = glob.glob(os.path.join(video_name_deg, "resize.mp4"))
        
        cap = cv2.VideoCapture(video[0])
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        interval = int(frame_count/30)
        
        frame_index = 0
        count_30 = 0
        
        
        if not cap.isOpened():
            print("Cannot open camera")
            exit()
        while True:
            
            ret, frame = cap.read()
            if not ret:
                print("Cannot receive frame")
                break
            keyName = cv2.waitKey(1)

            if (frame_index % interval) == 0 and count_30 < 30:
                if os.path.exists("../bbox_json/bbox.json"):
                    with open("../bbox_json/bbox.json", 'r') as json_file:
                        bbox = json.load(json_file)
                    if str(dataset_index) not in bbox:
                    
                        print(count_30)
                        
                        cv2.imwrite(os.path.join(data,"{index}.png".format(index = dataset_index)), frame)
                        
                        if count_30 == 0 or (dataset_index == len(bbox) and continue_flag == True):
                            continue_flag = False
                            area = cv2.selectROI('oxxostudio', frame, showCrosshair=False, fromCenter=False)
                            tracker.init(frame, area)    # 初始化追蹤器
                            tracking = True              # 設定可以開始
                        if tracking:
                            success, point = tracker.update(frame)   # 追蹤成功後，不斷回傳左上和右下的座標
                            if success:
                                frame_copy = frame.copy()
                                cv2.rectangle(frame_copy, (int(point[0]), int(point[1])), ( int(point[0]) + int(point[2]), int(point[1]) + int(point[3]) ), (0,0,255), 3)
                                cv2.imshow('oxxostudio', frame_copy)
                            keyName = cv2.waitKey(0)
                            if keyName == ord('a'):
                                area = cv2.selectROI('oxxostudio', frame, showCrosshair=False, fromCenter=False)
                                tracker = cv2.TrackerCSRT_create()
                                tracker.init(frame, area)
                                if tracking:
                                    success_new, point_new = tracker.update(frame)   # 追蹤成功後，不斷回傳左上和右下的座標
                                if success_new:
                                    cv2.rectangle(frame, (int(point[0]), int(point[1])), ( int(point[0]) + int(point[2]), int(point[1]) + int(point[3]) ), (0,0,255), 3)
                                    bbox[dataset_index] = (int(point[0]), int(point[1]), int(point[2]), int(point[3]))
                                    print(bbox[dataset_index])
                            else:
                                bbox[dataset_index] = (int(point[0]), int(point[1]), int(point[2]), int(point[3]))
                                print(bbox[dataset_index])


                                cv2.imshow('oxxostudio', frame)

                        with open("../bbox_json/bbox.json", 'w') as json_file:
                            json.dump(bbox, json_file, indent=4)
                    
                
            
                else:
                    if (frame_index % interval) == 0 and count_30 < 30:
                        print(count_30)

                        cv2.imwrite(os.path.join(data,"{index}.png".format(index = dataset_index)), frame)

                        if count_30 == 0:
                            area = cv2.selectROI('oxxostudio', frame, showCrosshair=False, fromCenter=False)
                            tracker.init(frame, area)    # 初始化追蹤器
                            tracking = True              # 設定可以開始
                        if tracking:
                            success, point = tracker.update(frame)   # 追蹤成功後，不斷回傳左上和右下的座標
                            if success:
                                frame_copy = frame.copy()
                                cv2.rectangle(frame_copy, (int(point[0]), int(point[1])), ( int(point[0]) + int(point[2]), int(point[1]) + int(point[3])), (0,0,255), 3)
                                cv2.imshow('oxxostudio', frame_copy)
                            keyName = cv2.waitKey(0)
                            if keyName == ord('a'):
                                area = cv2.selectROI('oxxostudio', frame, showCrosshair=False, fromCenter=False)
                                tracker = cv2.TrackerCSRT_create()
                                tracker.init(frame, area)
                                if tracking:
                                    success, point = tracker.update(frame)   # 追蹤成功後，不斷回傳左上和右下的座標
                                if success:
                                    cv2.rectangle(frame, (int(point[0]), int(point[1])), ( int(point[0]) + int(point[2]), int(point[1]) + int(point[3]) ), (0,0,255), 3)
                                    bbox[dataset_index] = (int(point[0]), int(point[1]), int(point[2]), int(point[3]))
                                    print(bbox[dataset_index])
                            else:
                                print(512)
                                bbox[dataset_index] = (int(point[0]), int(point[1]), int(point[2]), int(point[3]))
                                print(bbox[dataset_index])


                                cv2.imshow('oxxostudio', frame)

                        with open("../bbox_json/bbox.json", 'w') as json_file:
                                json.dump(bbox, json_file, indent=4)
                count_30 += 1
                dataset_index += 1
                    
            frame_index += 1
            
        cap.release()
        cv2.destroyAllWindows()
                



